In [1]:
#!/usr/bin/python
# needs shoutout to a couple of gits...

import logging
import os
import sys
import time
import argparse
from errno import *
from bioblend.galaxy import objects
from bioblend.galaxy.client import ConnectionError
from stat import S_IFDIR, S_IFREG, S_IFLNK
from fuse import Operations
from fuse import FUSE, FuseOSError

In [2]:
class GFSObject(Operations):

    def __init__(self, gfs):
        self.gfs = gfs

class GFSManager(Operations):
    context = None #should be abstract to force sub-class implementation
    
    def delegate(self, op, *args):

        boundObj = self._path_bound(args[0])
        logging.debug('{}.{} [{}]'.format(boundObj.__class__.__name__, op, args))

        if not hasattr(boundObj, op):
            raise FuseOSError(EFAULT)
            
        return getattr(boundObj, op)(*args)  

class GalaxyFS(Operations):
    
    def __init__(self, galaxy_url, api_key):
        self.gi = objects.GalaxyInstance(url=galaxy_url, api_key=api_key)
        self.root = RootDirectory()
        
        #TODO: for all subclasses of GFSObject register path, module grab classes inspect inheritance, b00m
        self.path_bindings = {
                                RootDirectory.context : self.root,
                                HistoryManager.context : HistoryManager(self),
                                ToolManager.context : ToolManager(self),
                                WorkflowManager.context : WorkflowManager(self)
                             }
    
    def _path_bound(self, path):
        pbits = path.split(os.path.sep) #TODO: path splitter function, maybe return enum?
        pbits_length = len(pbits)
        if pbits_length == 1: # ['']/
            bind = self.path_bindings.get(path)
        elif pbits_length > 1: # e.g. ['']/['histories']/['Unnamed History [8997977]']
            bind = self.path_bindings.get(pbits[1], False)       
            if not bind: # i.e. top level directory couldn't match
                bind = self.root # delegate handling to root directory
                
        return bind
    
    def __call__(self, op, *args):
        logging.debug('{} -> {} ::'.format(op, args[0]))
        boundObj = self._path_bound(args[0])
        return boundObj.delegate(op, *args)

In [3]:
class Directory(): #should subclass file? or fusepy object?
    
    def readdir(self, path=None, fh=None):
        return ['.','..']
    
    def getattr(self, path=None, fh=None):

        st = dict(st_mode=(S_IFDIR | 0700), st_nlink=2) #TODO: nlinks            
        # do we want actual create times?
        st['st_ctime'] = st['st_mtime'] = st['st_atime'] = time.time()
        st['st_uid'] = os.geteuid()
        st['st_gid'] = os.getegid()
        return st

class File():
    
    def getattr(self, path=None, fh=None):
        st = dict(st_mode=(S_IFREG | 0400), st_nlink=2) #TODO: nlinks            
        # do we want actual create times?
        st['st_ctime'] = st['st_mtime'] = st['st_atime'] = time.time()
        st['st_uid'] = os.geteuid()
        st['st_gid'] = os.getegid()
        return st

In [4]:
class RootDirectory(Directory, GFSManager):
    context = '/'
    tlds = ['histories', 'tools', 'workflows']#, 'libraries', 'tools', 'workflows']
    
    def _path_bound(self, path):
        return self
    
    def getattr(self, path=None, fh=None):
        
        if path == '/':
            st = dict(st_mode=(S_IFDIR | 0500), st_nlink=2) #TODO: nlinks            
            # do we want actual create times?
            st['st_ctime'] = st['st_mtime'] = st['st_atime'] = time.time()
            st['st_uid'] = os.geteuid()
            st['st_gid'] = os.getegid()
            return st
        else:
            raise FuseOSError(ENOENT)
            
    def readdir(self, path=None, fh=None):
        return RootDirectory.tlds+super(RootDirectory, self).readdir()

In [5]:
class ToolManager(GFSObject, GFSManager):
    context = 'tools'
    
    def stat_tools(self):
        self.eid = {}
        for tool in self.gfs.gi.tools.list():
            if '/' in tool.id:
                self.eid[tool.id.split('/')[-2]] = tool.id
            else:
                self.eid[tool.id] = tool.id
    
    def __init__(self, gfs):
        super(ToolManager, self).__init__(gfs)
        self.stat_tools()
        
    def _path_bound(self, path):
        wd = path.split(os.path.sep)
        wd_length = len(wd)
        if wd_length == 2: ##== /tools -> Tools
            return Tools(self)
        elif wd_length == 3: #
            return Tool(self.eid[wd[2]], self) ##== /tools/cut -> Tool
            
class Tools(Directory, GFSObject):
    
    def __init__(self, manager):
        self.manager = manager
        super(Tools, self).__init__(manager.gfs)

    def getattr(self, path=None, fh=None):
        if path == '/tools':
            return super(Tools, self).getattr(path, fh)
        else:
            raise FuseOSError(ENOENT)

    def readdir(self, path=None, fh=None):
        return self.manager.eid.keys()+super(Tools, self).readdir()
    
class Tool(File, GFSObject):
    
    def __init__(self, tool_id, manager):
        super(Tool, self).__init__(manager.gfs)
        self.manager = manager
        self.tool = self.gfs.gi.tools.get(tool_id)
    
        cal = []
        cal.append("#!/usr/bin/env python\n")
        cal.append("from bioblend.galaxy import objects\n")
        cal.append("import sys\n")
        cal.append("gi = objects.GalaxyInstance(url='%s', api_key='%s')\n" % (str(self.gfs.gi.gi.base_url), str(self.gfs.gi.gi.key)))
        #cal.append("dsetid = sys.argv[1][sys.argv[1].rfind('[')+1:sys.argv[1].rfind(']')]\n")
        #cal.append("print sys.argv[1].split('.')\n")
        cal.append("dsetid = sys.argv[1].split('/')[-1].split('.')[0]\n")
        cal.append("inputs = dict(src='hda', id=dsetid)\n")
        cal.append("print 'did: ',dsetid\n")
        cal.append("tool = gi.tools.get('%s')\n" % str(tool_id))
        cal.append("history = gi.histories.get('1cd8e2f6b131e891')\n")
        cal.append("result = tool.run(inputs, history)\n")
        cal.append("print result\n")

        self.back = ''.join(cal)
        
    def getattr(self, path=None, fh=None):
        st = dict(st_mode=(S_IFREG | 0500), st_nlink=2) #TODO: nlinks            
        # do we want actual create times?
        st['st_ctime'] = st['st_mtime'] = st['st_atime'] = time.time()
        st['st_uid'] = os.geteuid()
        st['st_gid'] = os.getegid()
        st['st_size'] = sys.getsizeof(self.back) #or should be this?  len(self.back)
        return st
    
    def read(self, path, size, offset, fh):  
        return self.back

In [6]:
class HistoryManager(GFSObject, GFSManager):
    context = 'histories'
    
    def __init__(self, gfs):
        super(HistoryManager, self).__init__(gfs)
        self.transactionMap = {}
    
    def _path_bound(self, path):
        if path in self.transactionMap:
            wd = self.transactionMap.get(path).split(os.path.sep)
            del self.transactionMap[path]
        else:
            wd = path.split(os.path.sep)
        
        wd_length = len(wd)
        if wd_length == 2: #== /histories -> Histories
            return Histories(self)
        elif wd_length == 3: #== /histories/Unnamed History [8997977]/ -> History
            hist_id = wd[2][wd[2].rfind('[')+1:wd[2].rfind(']')]
            try:
                return History(hist_id, self.gfs)
            except ConnectionError:
                return Histories(self)
        elif wd_length == 4: #== /histories/Unnamed History [8997977]/Pasted Entry [5969b1f7201f12ae] -> HistoryDataset
            hist_id = wd[2][wd[2].rfind('[')+1:wd[2].rfind(']')]
            dataset_id = wd[3][wd[3].rfind('[')+1:wd[3].rfind(']')]
            return History(hist_id, self.gfs).get_dataset(dataset_id)

class Histories(Directory, GFSObject):
    
    def __init__(self, manager):
        self.manager = manager
        super(Histories, self).__init__(manager.gfs)
    
    def getattr(self, path=None, fh=None):
        if path == '/histories':
            return super(Histories, self).getattr(path, fh)
        else:
            raise FuseOSError(ENOENT)
            
    def readdir(self, path=None, fh=None): # TODO: for all id's that go [inside], create tag function
        return [history.name+' ['+history.id+']' for history in self.gfs.gi.histories.list()]+super(Histories, self).readdir()
    
    def mkdir(self, path=None, mode=None):
        new_hist = self.gfs.gi.histories.create(path[path.rfind(os.path.sep)+1:])
        self.manager.transactionMap[path] = '/histories/{} [{}]'.format(new_hist.name, new_hist.id)
    
class History(Directory, GFSObject):

    def __init__(self, hist_id, gfs):
        super(History, self).__init__(gfs)
        self.hist = self.gfs.gi.histories.get(hist_id)
        
    def rmdir(self, path):
        self.gfs.gi.histories.delete(self.hist.id)

    def rename(self, old, new):
        path = new[:new.rfind(os.path.sep)]
        
        if path != '/histories':
            raise FuseOSError(EPERM)
        name = new[new.rfind(os.path.sep)+1:]
        self.hist.update(name=name)

    def readdir(self, path=None, fh=None): #TODO: tag function
        return ['{}. {} [{}]'.format(hist_item.wrapped['hid'], hist_item.name, hist_item.id) for hist_item in self.hist.content_infos if not hist_item.deleted]+super(History, self).readdir()
    
    def get_dataset(self, dataset_id):
        return HistoryDataset(self.hist, dataset_id, self.gfs)
    
class HistoryDataset(File, GFSObject):
    
    def __init__(self, hist, dataset_id, gfs):
        super(HistoryDataset, self).__init__(gfs)
        self.dataset = hist.get_dataset(dataset_id)
        
    def unlink(self, path):
        self.dataset.deleted()

In [7]:
class WorkflowManager(GFSObject, GFSManager):
    context = 'workflows'
    
    def __init__(self, gfs):
        super(WorkflowManager, self).__init__(gfs)
        self.transactionMap = {}
    
    def _path_bound(self, path):
        if path in self.transactionMap:
            wd = self.transactionMap.get(path).split(os.path.sep)
            del self.transactionMap[path]
        else:
            wd = path.split(os.path.sep)
        
        wd_length = len(wd)
        if wd_length == 2: #== /workflows -> Workflows
            return Workflows(self)
        elif wd_length == 3: #== /workflows/Unnamed Workflow [8997977]/ -> Workflow
            workf_id = wd[2][wd[2].rfind('[')+1:wd[2].rfind(']')]
            try:
                return Workflow(workf_id, self.gfs)
            except ConnectionError:
                return Workflows(self)
        ''' not sure needed yet
        elif wd_length == 4: #== /histories/Unnamed History [8997977]/Pasted Entry [5969b1f7201f12ae] -> HistoryDataset
            hist_id = wd[2][wd[2].rfind('[')+1:wd[2].rfind(']')]
            dataset_id = wd[3][wd[3].rfind('[')+1:wd[3].rfind(']')]
            return History(hist_id, self.gfs).getDataset(dataset_id)
        '''
class Workflows(Directory, GFSObject):
    
    def __init__(self, manager):
        self.manager = manager
        super(Workflows, self).__init__(manager.gfs)
    
    def getattr(self, path=None, fh=None):
        if path == '/workflows':
            return super(Workflows, self).getattr(path, fh)
        else:
            raise FuseOSError(ENOENT)
            
    def readdir(self, path=None, fh=None): # TODO: for all id's that go [inside], create tag function
        return [workflow.name+' ['+workflow.id+']' for workflow in self.gfs.gi.workflows.list()]+super(Workflows, self).readdir()
    
    ''' not yet
    def mkdir(self, path=None, mode=None):
        new_workf = self.gfs.gi.histories.create(path[path.rfind(os.path.sep)+1:])
        self.manager.transactionMap[path] = '/histories/{} [{}]'.format(new_hist.name, new_hist.id)
    '''
class Workflow(Directory, GFSObject):

    def __init__(self, workf_id, gfs):
        super(Workflow, self).__init__(gfs)
        self.workf = self.gfs.gi.workflows.get(workf_id)
        
    def rmdir(self, path):
        self.gfs.gi.workflows.delete(self.workf.id)

    '''
    def rename(self, old, new):
        path = new[:new.rfind(os.path.sep)]
        
        if path != '/histories':
            raise FuseOSError(EPERM)
        name = new[new.rfind(os.path.sep)+1:]
        self.hist.update(name=name)
    '''
    
    def readdir(self, path=None, fh=None): #TODO: tag function
        return super(Workflow, self).readdir() #['{}. {} [{}]'.format(hist_item.wrapped['hid'], hist_item.name, hist_item.id) for hist_item in self.hist.content_infos if not hist_item.deleted]+super(History, self).readdir()
    '''
    def getDataset(self, dataset_id):
        return HistoryDataset(self.hist, dataset_id, self.gfs)
    '''

In [9]:
logging.getLogger().setLevel(logging.WARNING)
gfs = GalaxyFS('http://localhost:8080', 'admin')

In [11]:
tool = gfs.gi.tools.get('rbc_rnabob')
for x in gfs.gi.gi.tools.show_tool(tool.id, True, False)['inputs']:
    print x['name']+"\t: "+x['label']+" ("+x['type']+")"


ERROR:bioblend:GET: error 404: '{"err_msg": "Could not find tool with id \'rbc_rnabob\'", "err_code": 404001}', 0 attempts left


ConnectionError: GET: error 404: '{"err_msg": "Could not find tool with id \'rbc_rnabob\'", "err_code": 404001}', 0 attempts left: None

In [46]:
tool = gfs.gi.tools.get('tp_tac')
history = gfs.gi.histories.get('1cd8e2f6b131e891')
inputs = dict(infile=dict(src='hda',id='03501d7626bd192f'))
print inputs
tool.run(inputs, history)

{'infile': {'src': 'hda', 'id': '03501d7626bd192f'}}


[HistoryDatasetAssociation({u'accessible': True, u'resubmitted': False, u'create_time': u'2015-05-05T14:45:43.639170', u'file_size': 0, u'dataset_id': u'1c84aa7fc4490e6d', u'id': u'fb85969571388350', u'misc_info': u'', u'hda_ldda': u'hda', u'download_url': u'/api/histories/1cd8e2f6b131e891/contents/fb85969571388350/display', u'state': u'queued', u'display_types': [], u'display_apps': [], u'type': u'file', u'file_path': None, u'misc_blurb': u'queued', u'peek': None, u'update_time': u'2015-05-05T14:45:44.005637', u'data_type': u'galaxy.datatypes.data.Text', u'tags': [], u'deleted': False, u'history_id': u'1cd8e2f6b131e891', u'meta_files': [], u'genome_build': u'?', u'hid': 18, u'visualizations': [], u'metadata_data_lines': 4, u'file_ext': u'txt', u'annotation': None, u'metadata_dbkey': u'?', u'history_content_type': u'dataset', u'name': u'tac on data 4', u'extension': u'txt', u'visible': True, u'url': u'/api/histories/1cd8e2f6b131e891/contents/fb85969571388350', u'uuid': u'ec7d8915-979d-

In [81]:
logging.getLogger().setLevel(logging.WARNING)
fuse = FUSE(gfs, '/home/cam/bench/gfs', foreground=True, nothreads=True, ro=False)